In [ ]:
# General Imports
import os
import numpy as np
import subprocess
import sys
import zipfile
import requests
from io import BytesIO

# Data
import pathlib
from pathlib import Path
import tifffile

# Architecture imports
import torch
import torch.nn as nn

from typing import List, Tuple
import shutil

from scipy.ndimage import zoom
from skimage import exposure
import warnings
from skimage.morphology import disk, binary_closing,binary_opening
from scipy import ndimage
import cc3d

In [ ]:
# Check if GPU is there
if torch.cuda.is_available():
    print("GPU available for execution ")
else:
    print("GPU not available ! . Please check or proceed to execute in CPU")

In [ ]:
input_path = Path(r"sample_test_Data/Input")
output_path = r"sample_test_Data/Output"

## Check Model Presence or Download 

In [ ]:
MODEL_DIR=r"BV_models"
ZIP_URL = "https://zenodo.org/records/15092730/files/BV_models.zip?download=1"

In [ ]:
# Model folder check
def download_and_extract_zip(url: str, extract_to: str):
    print(f"Downloading model zip from {url}...")
    response = requests.get(url)
    response.raise_for_status()  # raise error if download fails

    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        print(f"Extracting models to '{extract_to}'...")
        zip_ref.extractall(extract_to)
    print("Models downloaded and extracted successfully.")

def ensure_model_dir():
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR, exist_ok=True)
        download_and_extract_zip(ZIP_URL, MODEL_DIR)
    else:
        print(f"Model directory '{MODEL_DIR}' already exists. Skipping download.")

In [ ]:
ensure_model_dir() # If model files are not there it will begin download automatically 

In [ ]:
# Set model environment
os.environ["nnUNet_raw"] = "BV_models/BV_models/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "BV_models/BV_models/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "BV_models/BV_models/nnUNet_results"

In [ ]:
# nnunet details
dataset_num=111
config="3d_fullres"
on_demand=False

## Chunk the Image and Execute the model

In [ ]:
def make_chuncks(volume, output_folder, tif_file,chunk_size=(128, 256, 256)):
    # Calculate number of chunks in each dimension
    chunks_z = int(np.ceil(volume.shape[0] / chunk_size[0]))
    chunks_y = int(np.ceil(volume.shape[1] / chunk_size[1]))
    chunks_x = int(np.ceil(volume.shape[2] / chunk_size[2]))
    chunk_num=0

    for z in range(chunks_z):
        for y in range(chunks_y):
            for x in range(chunks_x):
                # Calculate chunk boundaries
                z_start, z_end = z * chunk_size[0], min((z + 1) * chunk_size[0], volume.shape[0])
                y_start, y_end = y * chunk_size[1], min((y + 1) * chunk_size[1], volume.shape[1])
                x_start, x_end = x * chunk_size[2], min((x + 1) * chunk_size[2], volume.shape[2])

                # Extract chunks
                volume_chunk = volume[z_start:z_end, y_start:y_end, x_start:x_end]
               
                # Pad chunks if necessary
                if volume_chunk.shape != chunk_size:
                    volume_chunk = np.pad(volume_chunk, 
                                          ((0, chunk_size[0] - volume_chunk.shape[0]), 
                                           (0, chunk_size[1] - volume_chunk.shape[1]), 
                                           (0, chunk_size[2] - volume_chunk.shape[2])),
                                          mode='constant')

                # Save chunks
                chunk_name = f"{tif_file[:-4]}_z{z}_y{y}_x{x}_{chunk_num:03}_0000.tif"
                tifffile.imwrite(output_folder / chunk_name, volume_chunk)
                chunk_num+=1
               
    print("chunks created ...")

In [ ]:
def resize_volume_bicubic(volume, target_size=(256, 819)):
    """
    Performs bicubic interpolation on a 3D volume array to resize x and y dimensions.
    
    Parameters:
    -----------
    volume : numpy.ndarray
        Input 3D volume with shape (z, y, x)
    target_size : tuple
        Desired output size for (y, x) dimensions, default is (256, 819)
        
    Returns:
    --------
    numpy.ndarray
        Resized volume with shape (z, 256, 819)
    """
    
    # Get current dimensions
    z_dim, y_dim, x_dim = volume.shape
    
    # Calculate zoom factors for each dimension
    z_factor = 1.0  # Keep z dimension unchanged
    y_factor = target_size[0] / y_dim
    x_factor = target_size[1] / x_dim
    
    # Perform bicubic interpolation
    # order=3 specifies bicubic interpolation
    resized_volume = zoom(volume, (z_factor, y_factor, x_factor), order=3)
    
    return resized_volume

In [ ]:
def restore_label_volume(label_volume, original_shape):
    """
    Resizes a label volume back to its original dimensions using nearest neighbor interpolation
    to preserve label values.
    
    Parameters:
    -----------
    label_volume : numpy.ndarray
        Input label volume with shape (z, 256, 819)
    original_shape : tuple
        Original shape to restore to (z, y, x)
        
    Returns:
    --------
    numpy.ndarray
        Restored label volume with original shape
    """
    
    # Get current dimensions
    z_dim, y_dim, x_dim = label_volume.shape
    
    # Calculate zoom factors for each dimension
    #z_factor = original_shape[0] / z_dim
    z_factor = 1
    y_factor = original_shape[1] / y_dim
    x_factor = original_shape[2] / x_dim
    
    # Use nearest neighbor interpolation (order=0) to preserve label values
    restored_volume = zoom(label_volume, (z_factor, y_factor, x_factor), order=0)
    
    return restored_volume

In [ ]:
def reconstruct_volume(chunks_folder, output_folder, final_shape, chunk_size=(128, 256, 256)):
    chunks_folder = Path(chunks_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    # Group chunks by original filename
    chunk_groups = {}
    for chunk_file in chunks_folder.glob("*.tif"):
        # Parse chunk file name based on the new pattern
        original_name, coords = chunk_file.stem.rsplit('_z', 1)
        z, yx_chunknum = coords.split('_y')
        y, x_chunknum = yx_chunknum.split('_x')
        x, chunk_num = x_chunknum.split('_')
        
        # Convert coordinates and chunk_num to integers
        z, y, x = int(z), int(y), int(x)
        
        # Group chunks by original file name
        if original_name not in chunk_groups:
            chunk_groups[original_name] = []
        chunk_groups[original_name].append((z, y, x, chunk_file))

    for original_name, chunks in chunk_groups.items():
        # Determine the shape of the padded volume
        max_z = max(chunk[0] for chunk in chunks) + 1
        max_y = max(chunk[1] for chunk in chunks) + 1
        max_x = max(chunk[2] for chunk in chunks) + 1

        # Initialize the reconstructed volume (padded)
        padded_shape = (
            max_z * chunk_size[0],
            max_y * chunk_size[1],
            max_x * chunk_size[2]
        )
        reconstructed_volume = np.zeros(padded_shape, dtype=np.float32)

        # Fill the reconstructed volume with chunks
        for z, y, x, chunk_file in chunks:
            chunk = tifffile.imread(chunk_file)
            reconstructed_volume[
                z * chunk_size[0] : (z + 1) * chunk_size[0],
                y * chunk_size[1] : (y + 1) * chunk_size[1],
                x * chunk_size[2] : (x + 1) * chunk_size[2]
            ] = chunk

        # Crop the reconstructed volume to the final shape
        #final_volume = reconstructed_volume[:final_shape[0], :final_shape[1], :final_shape[2]]
        final_volume = reconstructed_volume

        # Save the reconstruction
        #output_file = output_folder / f"{original_name}_reconstructed_original_before_shape_adjustment.tif"
        #tifffile.imwrite(output_file, final_volume)

        # Adjust the size of the final label
        restored_final_volume = restore_label_volume(final_volume, final_shape)

        # Unnecessary step - at this stage final shape should be the shaoe if restored_final_volume - but still clipping
        restored_final_volume = restored_final_volume[:final_shape[0], :final_shape[1], :final_shape[2]]

        # Save the reconstructed and cropped volume
        #output_file = output_folder / f"{original_name}_reconstructed.tif"
        #tifffile.imwrite(output_file, restored_final_volume)

    print("Reconstruction complete.")
    return restored_final_volume

In [ ]:
def run_nnunet(input_path, output_path, dataset_num, config):
    # Create output directory
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    # Run command
    cmd = [
        "nnUNetv2_predict",
        "-i", str(input_path),
        "-o", str(output_path),
        "-d", str(dataset_num),
        "-c", config,
        "--save_probabilities"
    ]

    result = ' '.join(cmd)
    print("command is", result)
    
    try:
        subprocess.run(cmd, check=True, text=True)
        print("Prediction completed successfully!")
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        sys.exit(1)

In [ ]:
def run_nnunet_ondemand(input_path, output_path, dataset_num, config):
    # Create output directory
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    # Run command
    cmd = [
        "nnUNetv2_predict",
        "-i", str(input_path),
        "-o", str(output_path),
        "-d", str(dataset_num),
        "-c", config,
        "--save_probabilities"
    ]

    cmd = ["conda", "run", "-p", ""] + cmd

    result = ' '.join(cmd)
    print("command is", result)
    
    try:
        subprocess.run(cmd, check=True, text=True)
        print("Prediction completed successfully!")
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        sys.exit(1)

In [ ]:
def execute_for_single_volume(file_path,output_path):
    print("Executing for ",file_path.stem)

    # Read the volume
    volume = tifffile.imread(file_path)
    
    # Make Chuncks
    chuncked_volume_output = Path(output_path) / (str(file_path.stem) + "_chunks")
    chuncked_volume_output.mkdir(parents=True, exist_ok=True)    
    make_chuncks(volume, chuncked_volume_output, file_path.name,chunk_size=(100, 819, 819))
    
    # Create Segmentation
    model_outputs = Path(output_path) / (str(file_path.stem) + "_segmentations")
    model_outputs.mkdir(parents=True, exist_ok=True)

    # Execute nn unet - change here if you running on downsampled image --- for faster execution remove downsampling in future
    if on_demand:
        run_nnunet_ondemand(chuncked_volume_output, model_outputs, dataset_num, config)
        #print("")
    else:
        run_nnunet(chuncked_volume_output, model_outputs, dataset_num, config)
    
    # Reconstruct
    restored_final_volume = reconstruct_volume(model_outputs, output_path, volume.shape, chunk_size=(100, 819, 819))
    return restored_final_volume

In [ ]:
def fill_small_regions(mask, original_image, max_diameter=20, intensity_threshold=0.05):
    """
    Fill holes in mask based on both size and intensity criteria.
    
    Args:
        mask (np.ndarray): Input 3D binary mask
        original_image (np.ndarray): Original intensity image
        max_diameter (int): Maximum diameter of regions to fill
        intensity_threshold (float): Minimum intensity threshold (as percentage)
        
    Returns:
        np.ndarray: Processed mask with holes filled based on criteria
    """
    # Get inverse of mask to identify holes
    inverse_mask = ~mask
    
    # Label connected components in the inverse mask (holes)
    labeled_holes, num_holes = ndimage.label(inverse_mask)
    
    # Calculate median intensity of all masked regions
    masked_intensities = original_image[mask]
    if len(masked_intensities) > 0:  # Check if any masked regions exist
        global_median = np.median(masked_intensities)
    else:
        global_median = 0
    
    # Initialize output mask
    filled_mask = mask.copy()
    
    # Process each hole
    for hole_label in range(1, num_holes + 1):
        # Extract current hole
        current_hole = labeled_holes == hole_label
        
        # Compute distance transform of the hole
        distances = ndimage.distance_transform_edt(current_hole)
        
        # Maximum diameter is twice the maximum distance from edge
        max_distance = np.max(distances)
        diameter = 2 * max_distance
        
        # Calculate mean intensity of the current hole
        hole_intensity = np.mean(original_image[current_hole])
        
        # Calculate intensity ratio compared to global median
        intensity_ratio = hole_intensity / global_median if global_median > 0 else 0
        
        # Fill hole if it meets both criteria:
        # 1. Diameter is less than threshold
        # 2. Intensity is at least n% (intensity_threshold) of the global median
        if diameter < max_diameter and intensity_ratio >= intensity_threshold:
            filled_mask[current_hole] = True
            
    return filled_mask

def make_watertight_with_size_constraint(mask, original_image, max_diameter=20, intensity_threshold=0.05):
    """
    Make a 3D binary mask watertight, filling holes based on size and intensity criteria.
    
    Args:
        mask (np.ndarray): Input 3D binary mask
        original_image (np.ndarray): Original intensity image
        max_diameter (int): Maximum diameter of regions to fill
        intensity_threshold (float): Minimum intensity threshold (as percentage)
        
    Returns:
        np.ndarray: Watertight 3D binary mask
    """
    # Create structure element for 3D connectivity
    struct = ndimage.generate_binary_structure(3, 1)
    
    # Step 1: Fill small holes in each 2D slice
    filled_2d = np.zeros_like(mask)
    for z in range(mask.shape[0]):
        slice_mask = mask[z]
        slice_image = original_image[z]
        filled_2d[z] = fill_small_regions(slice_mask, slice_image, max_diameter, intensity_threshold)
    
    # Step 2: Close small gaps
    closed = ndimage.binary_closing(filled_2d, structure=struct, iterations=1)
    
    # Step 3: Fill small holes in 3D for all components
    final_mask = fill_small_regions(closed, original_image, max_diameter, intensity_threshold)
    
    return final_mask

In [ ]:
def process_mask_with_size_constraint(mask_filepath, image_filepath, max_diameter=20, intensity_threshold=0.05):
    """
    Process a 3D mask file and make it watertight, filling holes based on size and intensity.
    
    Args:
        mask_filepath (str): Path to input mask TIFF file
        image_filepath (str): Path to original intensity image TIFF file
        max_diameter (int): Maximum diameter of regions to fill
        intensity_threshold (float): Minimum intensity threshold (as percentage)
        
    Returns:
        np.ndarray: Processed watertight 3D mask
    """
    # Read mask and original image
    mask = tifffile.imread(mask_filepath)
    original_image = tifffile.imread(image_filepath)
    
    # Ensure mask is binary
    if mask.dtype != bool:
        mask = mask > 0
    
    # Make watertight with size and intensity constraints
    watertight = make_watertight_with_size_constraint(
        mask, 
        original_image, 
        max_diameter, 
        intensity_threshold
    )
    
    return watertight

In [ ]:
def morphological_opening_3d_slices(binary_mask, disk_size=3):
    """
    Perform morphological closing on a 3D binary mask slice by slice using disk structure.
    
    Args:
        binary_mask (numpy.ndarray): 3D binary array
        disk_size (int): Diameter of the disk structure
    Returns:
        numpy.ndarray: Processed binary mask
    """
    # Create disk structure using skimage
    structure = disk(disk_size // 2)  # radius = diameter/2
    
    output = np.zeros_like(binary_mask)
    for i in range(binary_mask.shape[0]):
        output[i] = binary_opening(binary_mask[i], structure)
    
    return output

In [ ]:
def filter_objects_by_volume(mask, min_volume=None, max_volume=None):
    """
    Filter objects in a 3D binary mask based on their volume using connected components.
    
    Parameters:
    -----------
    mask : numpy.ndarray
        3D binary array where objects are marked with 1s and background with 0s
    min_volume : int or None
        Minimum volume threshold. Objects smaller than this will be removed.
        If None, no minimum threshold is applied.
    max_volume : int or None
        Maximum volume threshold. Objects larger than this will be removed.
        If None, no maximum threshold is applied.
        
    Returns:
    --------
    numpy.ndarray
        Filtered binary mask with same shape as input, where only objects
        meeting the volume criteria remain
    """
    
    # Run connected components
    labels = cc3d.connected_components(mask, connectivity=6)
    
    # Get volume of each component
    stats = cc3d.statistics(labels)
    volumes = stats['voxel_counts'][1:]  # Skip background (label 0)
    
    # Create mask for valid objects
    valid_objects = np.ones(len(volumes), dtype=bool)
    
    # Apply minimum volume threshold
    if min_volume is not None:
        valid_objects &= volumes >= min_volume
        
    # Apply maximum volume threshold
    if max_volume is not None:
        valid_objects &= volumes <= max_volume
    
    # Create output mask
    valid_labels = np.nonzero(valid_objects)[0] + 1  # Add 1 since we skipped background
    output_mask = np.isin(labels, valid_labels)
    
    return output_mask.astype(mask.dtype)

## Get all valid paths

In [ ]:
def process_icam2_paths(input_dir, output_dir):
    """
    Process directories to find Icam2 images and create corresponding result folders.
    
    Args:
        input_dir (str): Path to the input directory containing processed images
        output_dir (str): Path to create DAPI results folders
        
    Returns:
        Tuple[List[str], List[str]]: Lists of (input DAPI paths, output result folder paths)
    """
    # Initialize lists to store paths
    icam2_paths = []
    dapi_paths = []
    icam2_result_paths = []
    
    # Convert to Path objects for easier handling
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    # Create output directory if it doesn't exist
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(input_path):
        # Convert current root to Path object
        root_path = Path(root)
        
        # Check if we're in an isotropic_image folder
        if root_path.name == "isotropic_image":
            file_name = "C1-Icam2-Blood-Vessels.tif"
            # dapi_file_name = "C4-DAPI-XZ.tif"

            # Look for C4-DAPI-XZ.tif in files
            if file_name in files:
                # Get the full path to the DAPI XZ image
                full_path = root_path / file_name
                # adpi_path = root_path / dapi_file_name
                
                # Get the series folder name (parent of isotropic_image)
                series_folder = root_path.parent.name
                
                # Create corresponding output folder structure
                result_folder = output_path / series_folder / "ICAM2_results"
                result_folder.mkdir(parents=True, exist_ok=True)
                
                # Add paths to lists
                icam2_paths.append(str(full_path))
                # dapi_path = root_path / dapi_file_name
                icam2_result_paths.append(str(result_folder))
                
                print(f"Found  ICAM2 image: {full_path}")
                # print(f"Found DAPI image: {dapi_path}")
                print(f"Created results folder: {result_folder}")
    
    return icam2_paths, icam2_result_paths

In [ ]:
bv_paths, result_paths = process_icam2_paths(input_path, output_path)

In [ ]:
print(len(bv_paths))
print(bv_paths)

In [ ]:
for bv_input_file, result_output_directory in zip(bv_paths, result_paths):
    print("Executing BV  - model segmentation for :")
    print(bv_input_file)
    # Execute for single volume
    restored_final_volume = execute_for_single_volume(Path(bv_input_file),result_output_directory)

    # Save the nnunet output
    mask_output_file = Path(result_output_directory) / "C1-Icam2-Blood-Vessels_nnunet_reconstructed.tif"
    tifffile.imwrite(str(mask_output_file), restored_final_volume)

    # Clean up the segmentation
    # Process mask with size and intensity constraints
    opened_filtered = morphological_opening_3d_slices(restored_final_volume,disk_size=2)
    opened_output_file = Path(result_output_directory) / "C1-Icam2-Blood-Vessels_nnunet_reconstructed_opened.tif"
    tifffile.imwrite(str(opened_output_file), opened_filtered)
    
    result = process_mask_with_size_constraint(opened_output_file,bv_input_file,max_diameter=20,intensity_threshold=0.05)

    # Perform volume filtering
    volume_filtered = filter_objects_by_volume(result, min_volume=1000)
    
    # Save
    output_file = Path(result_output_directory) / "C1-Icam2-Blood-Vessels_nnunet_reconstructed_cleaned_filtered.tif"
    tifffile.imwrite(str(output_file), volume_filtered)

print("Processing complete !")